# Scenario 1 - Which countries have the most Invoices?
```bash
redis-cli -p 12000 FT.DROPINDEX idx
redis-cli -p 12000 FT.CREATE idx ON JSON PREFIX 1 invoice: SCHEMA $.BILLINGCOUNTRY AS BillingCountry TAG
while [[ $(redis-cli -p 12000 FT.INFO idx | awk '/percent_indexed/ {getline; print $0}') < 1 ]]; do sleep 1; done
redis-cli -p 12000 FT.AGGREGATE idx '*' GROUPBY 1 @BillingCountry REDUCE COUNT 0 AS Invoices SORTBY 2 @Invoices DESC MAX 5
```

In [18]:
! docker exec -it re1 /opt/scenarios/scenario_1.sh

OK
OK
1) (integer) 23
2) 1) "BillingCountry"
   2) "USA"
   3) "Invoices"
   4) "91"
3) 1) "BillingCountry"
   2) "Canada"
   3) "Invoices"
   4) "56"
4) 1) "BillingCountry"
   2) "France"
   3) "Invoices"
   4) "35"
5) 1) "BillingCountry"
   2) "Brazil"
   3) "Invoices"
   4) "35"
6) 1) "BillingCountry"
   2) "Germany"
   3) "Invoices"
   4) "28"


# Scenario 2 - Which city has the best customers?
```bash
redis-cli -p 12000 FT.DROPINDEX idx
redis-cli -p 12000 FT.CREATE idx ON JSON PREFIX 1 invoice: SCHEMA $.BILLINGCITY AS BillingCity TAG $.TOTAL AS Total NUMERIC
while [[ $(redis-cli -p 12000 FT.INFO idx | awk '/percent_indexed/ {getline; print $0}') < 1 ]]; do sleep 1; done
redis-cli -p 12000 FT.AGGREGATE idx '*' GROUPBY 1 @BillingCity REDUCE SUM 1 @Total AS InvoiceDollars SORTBY 2 @InvoiceDollars DESC MAX 3
```

In [19]:
! docker exec -it re1 /opt/scenarios/scenario_2.sh

OK
OK
1) (integer) 52
2) 1) "BillingCity"
   2) "Prague"
   3) "InvoiceDollars"
   4) "90.24"
3) 1) "BillingCity"
   2) "Paris"
   3) "InvoiceDollars"
   4) "77.24"
4) 1) "BillingCity"
   2) "Mountain View"
   3) "InvoiceDollars"
   4) "77.24"


# Scenario 3 - Who is the best customer?
```bash
redis-cli -p 12000 FT.DROPINDEX idx
redis-cli -p 12000 FT.CREATE idx ON JSON PREFIX 1 invoice: SCHEMA $.CUSTOMERID AS CustomerId NUMERIC $.TOTAL AS Total NUMERIC
while [[ $(redis-cli -p 12000 FT.INFO idx | awk '/percent_indexed/ {getline; print $0}') < 1 ]]; do sleep 1; done
redis-cli -p 12000 FT.AGGREGATE idx '*' GROUPBY 1 @CustomerId REDUCE SUM 1 @Total AS Money_Spent SORTBY 2 @Money_Spent DESC MAX 1
```


In [20]:
! docker exec -it re1 /opt/scenarios/scenario_3.sh

OK
OK
1) (integer) 58
2) 1) "CustomerId"
   2) "6"
   3) "Money_Spent"
   4) "49.62"


# Scenario 4 - Find the first 5 customers by Id who are not in the US.
```bash
redis-cli -p 12000 FT.DROPINDEX idx
redis-cli -p 12000 FT.CREATE idx ON JSON PREFIX 1 customer: SCHEMA $.COUNTRY AS Country TAG $.CUSTOMERID AS CustomerId NUMERIC SORTABLE
while [[ $(redis-cli -p 12000 FT.INFO idx | awk '/percent_indexed/ {getline; print $0}') < 1 ]]; do sleep 1; done
redis-cli -p 12000 FT.SEARCH idx "-@Country:{USA}" RETURN 4 CustomerId $.FIRSTNAME $.LASTNAME Country SORTBY CustomerId LIMIT 0 5
```

In [21]:
! docker exec -it re1 /opt/scenarios/scenario_4.sh

OK
OK
 1) (integer) 45
 2) "customer:CUSTOMERID:1"
 3) 1) "CustomerId"
    2) "1"
    3) "$.FIRSTNAME"
    4) "Lu\xef\xbf\xbd\xef\xbf\xbds"
    5) "$.LASTNAME"
    6) "Gon\xef\xbf\xbd\xef\xbf\xbdalves"
    7) "Country"
    8) "Brazil"
 4) "customer:CUSTOMERID:2"
 5) 1) "CustomerId"
    2) "2"
    3) "$.FIRSTNAME"
    4) "Leonie"
    5) "$.LASTNAME"
    6) "K\xef\xbf\xbd\xef\xbf\xbdhler"
    7) "Country"
    8) "Germany"
 6) "customer:CUSTOMERID:3"
 7) 1) "CustomerId"
    2) "3"
    3) "$.FIRSTNAME"
    4) "Fran\xef\xbf\xbd\xef\xbf\xbdois"
    5) "$.LASTNAME"
    6) "Tremblay"
    7) "Country"
    8) "Canada"
 8) "customer:CUSTOMERID:4"
 9) 1) "CustomerId"
    2) "4"
    3) "$.FIRSTNAME"
    4) "Bj\xef\xbf\xbd\xef\xbf\xbdrn"
    5) "$.LASTNAME"
    6) "Hansen"
    7) "Country"
    8) "Norway"
10) "customer:CUSTOMERID:5"
11) 1) "CustomerId"
    2) "5"
    3) "$.FIRSTNAME"
    4) "Franti\xef\xbf\xbd\xef\xbf\xbdek"
    5) "$.LASTNAME"
    6) "Wichterlov\xef\xbf\xbd\xef\xbf\xbd"
    7) "Cou

# Scenario 5 - Which employees are Sales Agents?
```bash
redis-cli -p 12000 FT.DROPINDEX idx
redis-cli -p 12000 FT.CREATE idx ON JSON PREFIX 1 employee: SCHEMA $.TITLE AS Title TAG
while [[ $(redis-cli -p 12000 FT.INFO idx | awk '/percent_indexed/ {getline; print $0}') < 1 ]]; do sleep 1; done
redis-cli -p 12000 FT.SEARCH idx "@Title:{Sales Support Agent}" RETURN 2 $.LASTNAME $.FIRSTNAME
```

In [23]:
! docker exec -it re1 /opt/scenarios/scenario_5.sh

OK
OK
1) (integer) 3
2) "employee:EMPLOYEEID:3"
3) 1) "$.LASTNAME"
   2) "Peacock"
   3) "$.FIRSTNAME"
   4) "Jane"
4) "employee:EMPLOYEEID:4"
5) 1) "$.LASTNAME"
   2) "Park"
   3) "$.FIRSTNAME"
   4) "Margaret"
6) "employee:EMPLOYEEID:5"
7) 1) "$.LASTNAME"
   2) "Johnson"
   3) "$.FIRSTNAME"
   4) "Steve"


# Scenario 6 - What is the count of line items for Invoice ID 37?
```bash
redis-cli -p 12000 FT.DROPINDEX idx
redis-cli -p 12000 FT.CREATE idx ON JSON PREFIX 1 invoiceline: SCHEMA $.INVOICEID AS InvoiceId NUMERIC
while [[ $(redis-cli -p 12000 FT.INFO idx | awk '/percent_indexed/ {getline; print $0}') < 1 ]]; do sleep 1; done
redis-cli -p 12000 FT.AGGREGATE idx "@InvoiceId:[37 37]" GROUPBY 0 REDUCE COUNT 0 as Count
```

In [24]:
! docker exec -it re1 /opt/scenarios/scenario_6.sh

OK
OK
1) (integer) 1
2) 1) "Count"
   2) "4"


# Scenario 7 - Which artists have written the most Rock music?
```bash
redis-cli -p 12000 FT.DROPINDEX idx
redis-cli -p 12000 FT.CREATE idx ON JSON PREFIX 1 rockview: SCHEMA $.SONGS AS Songs NUMERIC SORTABLE
while [[ $(redis-cli -p 12000 FT.INFO idx | awk '/percent_indexed/ {getline; print $0}') < 1 ]]; do sleep 1; done
redis-cli -p 12000 FT.SEARCH idx '*' SORTBY Songs DESC LIMIT 0 10
```

In [25]:
! docker exec -it re1 /opt/scenarios/scenario_7.sh

OK
OK
 1) (integer) 51
 2) "rockview:ARTISTID:22"
 3) 1) "Songs"
    2) "114"
    3) "$"
    4) "{\"ARTISTID\":22,\"NAME\":\"Led Zeppelin\",\"SONGS\":114}"
 4) "rockview:ARTISTID:150"
 5) 1) "Songs"
    2) "112"
    3) "$"
    4) "{\"ARTISTID\":150,\"NAME\":\"U2\",\"SONGS\":112}"
 6) "rockview:ARTISTID:58"
 7) 1) "Songs"
    2) "92"
    3) "$"
    4) "{\"ARTISTID\":58,\"NAME\":\"Deep Purple\",\"SONGS\":92}"
 8) "rockview:ARTISTID:90"
 9) 1) "Songs"
    2) "81"
    3) "$"
    4) "{\"ARTISTID\":90,\"NAME\":\"Iron Maiden\",\"SONGS\":81}"
10) "rockview:ARTISTID:118"
11) 1) "Songs"
    2) "54"
    3) "$"
    4) "{\"ARTISTID\":118,\"NAME\":\"Pearl Jam\",\"SONGS\":54}"
12) "rockview:ARTISTID:152"
13) 1) "Songs"
    2) "52"
    3) "$"
    4) "{\"ARTISTID\":152,\"NAME\":\"Van Halen\",\"SONGS\":52}"
14) "rockview:ARTISTID:51"
15) 1) "Songs"
    2) "45"
    3) "$"
    4) "{\"ARTISTID\":51,\"NAME\":\"Queen\",\"SONGS\":45}"
16) "rockview:ARTISTID:142"
17) 1) "Songs"
    2) "41"
    3) "$"
    4) "{